In [1]:
import pandas as pd
import torch
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments
)
from datasets import Dataset
from sklearn.metrics import f1_score, classification_report
import numpy as np

/Users/george/Documents/projects/LCT_2025/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Data

In [2]:
df = pd.read_pickle("../data/df_final_v2.pkl")

In [3]:
df = df.drop(columns = ['predicted_classes_new'])

In [4]:
df.head(3)

,index,id,title,text,dateCreate,product,sub_product,source,text_clean,review,predicted_classes_refined,final_classes
0,0,992651,не выполняют условия акции,в апреле 2025 года я рекомендовала дебетовую к...,2025-08-29T23:30:38.746003Z,debitCards,Умная карта (Премиум),sravni,в апреле года я рекомендовала дебетовую карту ...,не выполняют условия акции в апреле года я ре...,"[1, 3, 14, 15]","[debitCards, cashbackPromo, notifications, ref..."
1,1,998360,жалоба на услугу газпром бонус премиум,купил услугу газпром бонус премиум за 2 цель б...,2025-09-15T09:38:13.34818Z,debitCards,Отсутствует,sravni,купил услугу газпром бонус премиум за цель был...,жалоба на услугу газпром бонус премиум купил ...,[],[debitCards]
2,2,993744,банк не отвечает за слова своих сотрудников не...,хочу поделиться историей которая убила моё дов...,2025-09-02T23:21:16.507166Z,debitCards,Умная карта,sravni,хочу поделиться историей которая моё к газпром...,банк не отвечает за слова своих сотрудников не...,"[1, 6, 15]","[debitCards, cashbackPromo, earlyRepayment, re..."


In [5]:
df = df[['index', 'text', 'final_classes']].dropna().reset_index(drop=True)

In [19]:
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df['final_classes'])

df['labels'] = y.tolist()

print("Количество классов:", len(mlb.classes_))
print("Примеры меток:", mlb.classes_)

Количество классов: 23
Примеры меток: ['Salary&PremiumCards' 'autocredits' 'cardAccess' 'cashbackPromo'
 'creditCards' 'creditCardsService' 'credits' 'currencyExchange'
 'debitCards' 'depositAccess' 'deposits' 'earlyRepayment' 'loanIssues'
 'mortgage' 'mortgageIssues' 'notifications' 'other' 'refinancing'
 'remoteService' 'restructing' 'savings' 'serviceLevel'
 'transactionErrors']


In [7]:
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['text'], df['labels'], test_size=0.2, random_state=42
)

train_df = pd.DataFrame({'text': train_texts, 'labels': train_labels})
test_df = pd.DataFrame({'text': test_texts, 'labels': test_labels})

# Model

## Classic Train Test

In [8]:
#MODEL_NAME = "DeepPavlov/rubert-base-cased"
# MODEL_NAME = "DeepPavlov/rubert-base"
MODEL_NAME = "ai-forever/ruBert-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=256
    )

/home/gleb/LCT_2025/.venv/lib/python3.13/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

train_dataset = train_dataset.remove_columns(["text", "__index_level_0__"]) if "__index_level_0__" in train_dataset.column_names else train_dataset.remove_columns(["text"])
test_dataset = test_dataset.remove_columns(["text", "__index_level_0__"]) if "__index_level_0__" in test_dataset.column_names else test_dataset.remove_columns(["text"])

train_dataset.set_format("torch")
test_dataset.set_format("torch")

Map:   0%|          | 0/40285 [00:00<?, ? examples/s]

Map:   0%|          | 0/10072 [00:00<?, ? examples/s]

In [10]:
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(mlb.classes_),
    problem_type="multi_label_classification"
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ai-forever/ruBert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = (torch.sigmoid(torch.tensor(logits)) > 0.5).int().numpy()
    labels = labels
    return {
        "f1_micro": f1_score(labels, preds, average="micro"),
    }

In [12]:
def cast_labels_to_float(example):
    example["labels"] = [float(x) for x in example["labels"]]
    return example

train_dataset = train_dataset.map(cast_labels_to_float)
test_dataset = test_dataset.map(cast_labels_to_float)

train_dataset.set_format("torch")
test_dataset.set_format("torch")

Map:   0%|          | 0/40285 [00:00<?, ? examples/s]

Map:   0%|          | 0/10072 [00:00<?, ? examples/s]

In [ ]:
# training_args = TrainingArguments(
#     output_dir="../classifier_results",
#     eval_strategy="epoch",
#     save_strategy="epoch",
#     learning_rate=2e-5,
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=8,
#     num_train_epochs=3,
#     weight_decay=0.01,
#     logging_dir='../logs',
#     logging_steps=50,
#     load_best_model_at_end=True,
#     metric_for_best_model="f1_micro",
#     save_total_limit=2,
# )

In [13]:
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback

training_args = TrainingArguments(
    output_dir="../classifier_results",
    evaluation_strategy="epoch",       # оценка раз в эпоху
    save_strategy="epoch",             # сохраняем модель раз в эпоху
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=4,               # максимум 10 эпох
    weight_decay=0.01,
    logging_dir='../logs',
    logging_strategy="epoch",          # логируем раз в эпоху
    load_best_model_at_end=True,       # возвращаем лучшую модель
    metric_for_best_model="f1_micro",  # метрика для ранней остановки
    greater_is_better=True,
    save_total_limit=2
)

# callback для ранней остановки
early_stopping = EarlyStoppingCallback(early_stopping_patience=1)  
# patience=1 → если следующая эпоха не улучшила метрику, останавливаемся

/home/gleb/LCT_2025/.venv/lib/python3.13/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [14]:
train_dataset = train_dataset.map(lambda x: {"labels_float": np.array(x["labels"], dtype=np.float32)})
test_dataset  = test_dataset.map(lambda x: {"labels_float": np.array(x["labels"], dtype=np.float32)})

# Убираем оригинальные labels (чтобы Trainer не трогал их)
train_dataset = train_dataset.remove_columns("labels")
test_dataset  = test_dataset.remove_columns("labels")

# Переименовываем float колонку в labels
train_dataset = train_dataset.rename_column("labels_float", "labels")
test_dataset  = test_dataset.rename_column("labels_float", "labels")

# Теперь set_format
train_dataset.set_format("torch")
test_dataset.set_format("torch")

Map:   0%|          | 0/40285 [00:00<?, ? examples/s]

/tmp/ipykernel_307491/492247788.py:1: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  train_dataset = train_dataset.map(lambda x: {"labels_float": np.array(x["labels"], dtype=np.float32)})


Map:   0%|          | 0/10072 [00:00<?, ? examples/s]

/tmp/ipykernel_307491/492247788.py:2: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  test_dataset  = test_dataset.map(lambda x: {"labels_float": np.array(x["labels"], dtype=np.float32)})


In [15]:
import torch

# Проверяем, есть ли GPU
print(torch.cuda.is_available())  # True, если GPU доступен
print(torch.cuda.device_count())  # количество доступных GPU
print(torch.cuda.get_device_name(0))  # имя первого GPU


True
1
NVIDIA GeForce RTX 4070 Ti SUPER


In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(120138, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train() 

Epoch,Training Loss,Validation Loss,F1 Micro
1,0.158400,0.123810,0.726126
2,0.109600,0.108131,0.769384
3,0.089500,0.106506,0.777725
4,0.075800,0.105078,0.781371


TrainOutput(global_step=20144, training_loss=0.10831447325593617, metrics={'train_runtime': 2689.3523, 'train_samples_per_second': 59.918, 'train_steps_per_second': 7.49, 'total_flos': 2.120285478614016e+16, 'train_loss': 0.10831447325593617, 'epoch': 4.0})

In [18]:
predictions = trainer.predict(test_dataset)
preds = (torch.sigmoid(torch.tensor(predictions.predictions)) > 0.5).int().numpy()

print("\n--- Классификационный отчёт ---\n")
print(classification_report(test_labels.tolist(), preds, target_names=mlb.classes_))


--- Классификационный отчёт ---

                     precision    recall  f1-score   support

Salary&PremiumCards       0.79      0.73      0.76       892
        autocredits       0.78      0.68      0.72       788
         cardAccess       0.80      0.79      0.80       900
      cashbackPromo       0.83      0.91      0.87      1064
        creditCards       0.78      0.68      0.73      1669
 creditCardsService       0.79      0.72      0.75       856
            credits       0.79      0.71      0.75       814
   currencyExchange       0.77      0.44      0.56       557
         debitCards       0.86      0.87      0.87      5686
      depositAccess       0.79      0.65      0.71       626
           deposits       0.75      0.76      0.75       715
     earlyRepayment       0.83      0.83      0.83       906
         loanIssues       0.82      0.71      0.76       688
           mortgage       0.79      0.80      0.79       674
     mortgageIssues       0.84      0.85      0.84

/home/gleb/LCT_2025/.venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/gleb/LCT_2025/.venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


## Final Model (Full Data)

In [7]:
df

,index,text,final_classes,labels
0,0,в апреле 2025 года я рекомендовала дебетовую к...,"[debitCards, cashbackPromo, notifications, ref...","[1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
1,1,купил услугу газпром бонус премиум за 2 цель б...,[debitCards],"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
2,2,хочу поделиться историей которая убила моё дов...,"[debitCards, cashbackPromo, earlyRepayment, re...","[1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, ..."
3,3,в июне 2025 года я порекомендовал премиальную ...,"[debitCards, cashbackPromo, notifications, ref...","[0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
4,4,добрый день в связи с устройством на новую раб...,"[debitCards, depositAccess, remoteService]","[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, ..."
...,...,...,...,...
50352,50352,газпромбанк тут заманивал кредитом одобренным ...,"[other, earlyRepayment, loanIssues]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, ..."
50353,50353,ситуация довольно странная для входа в интерне...,[other],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
50354,50354,в банке нравится все и обслуживание на отлично...,[other],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
50355,50355,обращение в контакт центр 10 баллов а вот рабо...,[other],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [20]:
dataset = Dataset.from_pandas(df)

dataset = dataset.map(tokenize_function, batched=True)

dataset = dataset.map(lambda x: {"labels_float": np.array(x["labels"], dtype=np.float32)})
dataset = dataset.remove_columns("labels")
dataset = dataset.rename_column("labels_float", "labels")
dataset.set_format("torch")

Map:   0%|          | 0/50357 [00:00<?, ? examples/s]

Map:   0%|          | 0/50357 [00:00<?, ? examples/s]

In [23]:
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME, 
    num_labels=len(mlb.classes_),
    problem_type="multi_label_classification"
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ai-forever/ruBert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
training_args = TrainingArguments(
    output_dir="../classifier_full_model",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=4,           # только 4 эпохи
    weight_decay=0.01,
    logging_dir='../logs_full',
    logging_strategy="epoch",
    load_best_model_at_end=False,  # без early stopping
    save_total_limit=1
)


/home/gleb/LCT_2025/.venv/lib/python3.13/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [27]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer
)

# обучение
trainer.train()

Step,Training Loss
6295,0.100400
12590,0.086900
18885,0.069800
25180,0.056900


TrainOutput(global_step=25180, training_loss=0.07849992141541837, metrics={'train_runtime': 3477.2388, 'train_samples_per_second': 57.928, 'train_steps_per_second': 7.241, 'total_flos': 2.650396322367283e+16, 'train_loss': 0.07849992141541837, 'epoch': 4.0})

In [28]:
model.save_pretrained("../classifier_full_model")
tokenizer.save_pretrained("../classifier_full_model")

('../classifier_full_model/tokenizer_config.json',
 '../classifier_full_model/special_tokens_map.json',
 '../classifier_full_model/vocab.txt',
 '../classifier_full_model/added_tokens.json',
 '../classifier_full_model/tokenizer.json')

In [30]:
df

,text,final_classes,labels
0,в апреле 2025 года я рекомендовала дебетовую к...,"[debitCards, cashbackPromo, notifications, ref...","[1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
1,купил услугу газпром бонус премиум за 2 цель б...,[debitCards],"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
2,хочу поделиться историей которая убила моё дов...,"[debitCards, cashbackPromo, earlyRepayment, re...","[1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, ..."
3,в июне 2025 года я порекомендовал премиальную ...,"[debitCards, cashbackPromo, notifications, ref...","[0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
4,добрый день в связи с устройством на новую раб...,"[debitCards, depositAccess, remoteService]","[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, ..."
...,...,...,...
50352,газпромбанк тут заманивал кредитом одобренным ...,"[other, earlyRepayment, loanIssues]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, ..."
50353,ситуация довольно странная для входа в интерне...,[other],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
50354,в банке нравится все и обслуживание на отлично...,[other],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
50355,обращение в контакт центр 10 баллов а вот рабо...,[other],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


# Inference

In [8]:
MODEL_PATH = "../classifier_full_model"

In [17]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH)
model.eval()

# если доступен GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(120138, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [10]:
def predict_labels(texts, threshold=0.5):
    if isinstance(texts, str):
        texts = [texts]  # если один текст
    
    inputs = tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt"
    ).to(device)

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = torch.sigmoid(logits).cpu().numpy()

    # бинаризация по порогу
    preds = (probs > threshold).astype(int)

    # преобразуем обратно в имена меток
    predicted_labels = [mlb.inverse_transform([p])[0] for p in preds]
    return predicted_labels

In [23]:
mlb.classes_

array(['Salary&PremiumCards', 'autocredits', 'cardAccess',
       'cashbackPromo', 'creditCards', 'creditCardsService', 'credits',
       'currencyExchange', 'debitCards', 'depositAccess', 'deposits',
       'earlyRepayment', 'loanIssues', 'mortgage', 'mortgageIssues',
       'notifications', 'other', 'refinancing', 'remoteService',
       'restructing', 'savings', 'serviceLevel', 'transactionErrors'],
      dtype=object)

In [24]:
import pickle
with open(f'{MODEL_PATH}/final_mlb_encoder.pkl', 'wb') as f:
    pickle.dump(mlb, f)

In [28]:
import pickle
with open(f'{MODEL_PATH}/final_mlb_encoder.pkl', 'rb') as f:
    mlb = pickle.load(f)

In [11]:
df

,index,text,final_classes,labels
0,0,в апреле 2025 года я рекомендовала дебетовую к...,"[debitCards, cashbackPromo, notifications, ref...","[1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
1,1,купил услугу газпром бонус премиум за 2 цель б...,[debitCards],"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
2,2,хочу поделиться историей которая убила моё дов...,"[debitCards, cashbackPromo, earlyRepayment, re...","[1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, ..."
3,3,в июне 2025 года я порекомендовал премиальную ...,"[debitCards, cashbackPromo, notifications, ref...","[0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
4,4,добрый день в связи с устройством на новую раб...,"[debitCards, depositAccess, remoteService]","[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, ..."
...,...,...,...,...
50352,50352,газпромбанк тут заманивал кредитом одобренным ...,"[other, earlyRepayment, loanIssues]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, ..."
50353,50353,ситуация довольно странная для входа в интерне...,[other],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
50354,50354,в банке нравится все и обслуживание на отлично...,[other],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
50355,50355,обращение в контакт центр 10 баллов а вот рабо...,[other],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [12]:
def predict_labels(texts, threshold=0.5):
    if isinstance(texts, str):
        texts = [texts]  # если один текст
    
    inputs = tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt"
    ).to(device)

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = torch.sigmoid(logits).cpu().numpy()

    # бинаризация по порогу
    preds = (probs > threshold).astype(int)
    
    # теперь preds точно numpy array с формой (num_texts, num_labels)
    predicted_labels = mlb.inverse_transform(preds)
    
    return list(predicted_labels)

In [ ]:
inputs = tokenizer(
    texts,
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors="pt"
).to(device)

In [43]:
l = [1,2,3]
l.pop(0)
l

[2, 3]

In [35]:
import json
texts = df.sample(10)['text'].tolist()
json.dumps({
    'data': [{'id': i, 'text': text} for i, text in enumerate(texts)]
}, ensure_ascii=False)

'{"data": [{"id": 0, "text": "я брал кредит уже выплатил пользуюсь картами банка у меня нет никаких нареканий по поводу работы и обслуживания мне очень нравится работа банка хочется пожелать всем работникам газпромбанка здоровья и успехов "}, {"id": 1, "text": "недавно обратился в газпромбанк и был ужасно удивлён разгильдяйским отношением банка к клиенту br 22 07 оставил заявку на кредитную карту банк одобрил заявку на кредитную карту на следующий день со мной связался сотрудник банка с которым мы проверили данные и согласовали время для приезда курьера 24 07 ко мне приехал курьер для подписания договора на кредитную карту но кто то в офисе перепутал вложения и в конверте с моим именем оказался договор на совершенно другого человека мы с курьером договорились что он привезёт мой договор через день согласовали время и спокойно попрощались как вечером этого же дня мне поступает смс от газпром банка что по моей заявке принято отрицательное решение сказать что я удивился ничего не сказать 

In [31]:
predict_labels(texts)

[('restructing',),
 ('Salary&PremiumCards', 'debitCards'),
 ('debitCards',),
 ('creditCards', 'creditCardsService', 'currencyExchange', 'earlyRepayment'),
 ('Salary&PremiumCards', 'debitCards', 'transactionErrors'),
 ('creditCards',),
 ('debitCards',),
 ('creditCards',),
 ('Salary&PremiumCards', 'debitCards', 'depositAccess', 'deposits'),
 ('debitCards',)]

In [ ]:
batch_size = 8
all_preds = []

for i in range(0, len(df), batch_size):
    batch_texts = df['text'].iloc[i:i+batch_size].tolist()
    batch_preds = predict_labels(batch_texts)
    all_preds.extend(batch_preds)

df['predicted_classes'] = all_preds

IndexError: list index out of range

In [54]:
df['predicted_classes'].value_counts().sum()

np.int64(50357)

In [56]:
df.to_csv('../data/final_dataset/classification_df.csv', index = False)

In [ ]:
example_text = "Этот фильм был просто невероятный, я в восторге!"
predicted = predict_labels(example_text)
print(predicted)
# например -> ['positive', 'emotion_happy']